In [198]:
import pandas as pd
import numpy as np
import requests
from sklearn.neighbors import KDTree, BallTree
from tqdm import tqdm

## Data Loading

In [163]:
pop = pd.read_csv('population_points.csv')
charging_station = pd.read_csv('public charging station.csv')
pop.info()
print()
charging_station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28686 entries, 0 to 28685
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   NTL_value  28686 non-null  float64
 1   xcoord     28686 non-null  float64
 2   ycoord     28686 non-null  float64
dtypes: float64(3)
memory usage: 672.5 KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4477 entries, 0 to 4476
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   No.        4477 non-null   int64  
 1   Name       4477 non-null   object 
 2   Webpage    4477 non-null   object 
 3   Address    4477 non-null   object 
 4   Brand      4477 non-null   object 
 5   Num_of_DC  4477 non-null   int64  
 6   Num_of_AC  4477 non-null   int64  
 7   Type       4477 non-null   object 
 8   lng        4477 non-null   float64
 9   lat        4477 non-null   float64
dtypes: float64(2), int64(3), object(5)
memory usage: 349.9+ K

## AutoNavi Map API

In [8]:
API_KEY = 'db724554305fa4a8a31ab450cc6657ed'

In [279]:
def get_driving_distance(origins, destination, API_KEY):
    url = 'https://restapi.amap.com/v3/distance?'
    
    #Param origin
    origin_str = []
    for tp in origins:
        origin_str.append(f'{tp[0]},{tp[1]}')
    origin_str = 'origins=' + '|'.join(origin_str)
    
    #Param destination
    dest_str = f'destination={destination[0]},{destination[1]}'
    
    #Param type('1' indicates calculating driving distance)
    type_str = 'type=1'
    
    #Param key
    key_str = f'key={API_KEY}'
    
    url = '&'.join([url, origin_str, dest_str, type_str, key_str])
#     print(url)
    
    ret = requests.get(url)
    
    if ret.status_code == 200:
        response = ret.json()
        if response['status'] == '0':
            return False, response['info']
        else:
            try:
                return True, response['results'] # returns a list of json data
            except KeyError:
                print(response)
    else:
        print('404, Please check what happened.')
        return False, url

In [115]:
status, dist = get_driving_distance([(121.48134674,31.21772284)],(121.4760268,31.23207944), API_KEY)

In [116]:
dist

[{'origin_id': '1', 'dest_id': '1', 'distance': '2136', 'duration': '534'}]

## Finding CS-Pop pairs and Create mappings

For each charging station, we need to construct a mapping from charging_station_id to a list of population point ids. The corresponding population points of a certain charging station should lie within a d-km circle. This reduces unnecessary API requests.

In [374]:
THRESHOLD = 2000 # meters
EARTH_RADIUS = 6371000

In [304]:
# Creating ball tree for near neighbors
pop_array = pop[['ycoord','xcoord']].values * np.pi / 180
bTree = BallTree(pop_array, leaf_size=30000, metric='haversine')

In [305]:
# Creating mapping from charging station to population points

cs_coords_rad = charging_station[['lat','lng']].values * np.pi / 180

cs2pop = {}
for i in range(charging_station.shape[0]):
    cs2pop[i] = []
    
for id_, pair in enumerate(cs_coords_rad):
    id_array = bTree.query_radius(pair.reshape(1,-1), r=THRESHOLD/6371000)[0].tolist()
    cs2pop[id_] = id_array

In [306]:
# Inversely, create mapping from population points to charging stations
pop2cs = {}
for i in range(pop.shape[0]):
    pop2cs[i] = []

for cs_id, pop_id_list in cs2pop.items():
    for pop_id in pop_id_list:
        pop2cs[pop_id].append(cs_id)

## Improved Gravity Model

$$A_i = \sum_{j=1}^{n}\frac{M_j}{D_{ij}^\theta V_j}$$
$$V_j = \sum_{i=1}^{m}\frac{P_i}{D_{ij}^\gamma}$$
$$M_j = \alpha AC_j + \beta DC_j$$

### Precomputing the distance pairs using AutoNavi Map API

In [307]:
cs_coords = charging_station[['lng','lat']].values
pop_coords = pop[['xcoord','ycoord']].values

In [310]:
sum_ = 0
dist_list = []
for id, lst in cs2pop.items():
    sum_ += len(lst)
    dist_list.append(len(lst))
print(sum_)
print(sum_/len(cs2pop))
print(max(dist_list))
print(min(dist_list))

1033320
230.80634353361626
240
100


In [309]:
sum__ = 0
cnt_ = 0
for id, lst in pop2cs.items():
    if len(lst) != -1:
        sum__ += len(lst)
        cnt_ += 1
print(sum__)
print(sum__/cnt_)

1033320
36.02175277138674


In [318]:
pre_computed_distance = {}
checkpoints = []

In [319]:
# Make requests to the API for computing distance and saving those distance into a dictionary
# 1st recored at 2:30pm, Jul 16, Sat

for cs_id, pop_id_list in tqdm(cs2pop.items()):
    # In case the program stops halfway due to request limits per day, we have a checkpoint list to record how many points we have done
    if cs_id in checkpoints:
        continue
    
    # Retrieve coordinates from ids
    destination_coords = cs_coords[cs_id]
    pop_coords_list = []
    for pop_id in pop_id_list:
        pop_coords_list.append(pop_coords[pop_id])

    # Make request
    res_json = []
    for i in range((len(pop_id_list) - 1) // 100 + 1):
        cur_pop_coords_list = pop_coords_list[i*100:(i+1)*100]
        cur_request_flag, cur_res_json = get_driving_distance(cur_pop_coords_list, destination_coords, API_KEY)
        res_json += cur_res_json
    # See if the request has went through
    if request_flag:
        # Just in case something went wrong with the response from the API
        assert len(pop_id_list) == len(res_json)
        
        # Save the computed data into the dictionary and record in checkpoints
        for i in range(len(pop_id_list)):
            pre_computed_distance[(pop_id_list[i], cs_id)] = res_json[i]["distance"]
        checkpoints.append(cs_id)
    else:
        print(res_json)

100%|██████████| 4477/4477 [44:36<00:00,  1.67it/s] 


In [320]:
#saving the precomputed distance pairs
import pickle

with open('precomputed_dist_pairs.pickle', 'wb') as file:
    pickle.dump(pre_computed_distance, file, protocol=pickle.HIGHEST_PROTOCOL)

In [373]:
cs2pop_final_4km = {}

In [375]:
# Re-filter the population points around a CS with the distance calculated by the API
for cs_id, pop_id_list in cs2pop.items():
    new_pop_id_list = []
    for pop_id in pop_id_list:
        if int(pre_computed_distance[(pop_id, cs_id)]) <= THRESHOLD:
            new_pop_id_list.append(pop_id)
    cs2pop_final_4km[cs_id] = new_pop_id_list

In [376]:
# Same as before, inversely create a mapping from population point id to charging station id
pop2cs_final_4km = {}
for i in range(pop.shape[0]):
    pop2cs_final_4km[i] = []

for cs_id, pop_id_list in cs2pop_final_4km.items():
    for pop_id in pop_id_list:
        pop2cs_final_4km[pop_id].append(cs_id)

In [377]:
# Checking how many charging stations are within the threshold distance from a population point on average
sum__ = 0
cnt_ = 0
for id, lst in pop2cs_final_4km.items():
    if len(lst) != -1:
        sum__ += len(lst)
        cnt_ += 1
print(sum__)
print(sum__/cnt_)

81825
2.8524367287178416


**Threshold caliberating:**
<br><br>
1km - 0.52 charging station per population point <br>
**2km - 2.85 charging stations per population point <br>**
3km - 7.53 charging stations per population point <br>
4km - 14.70 charging stations per population point<br>

In [378]:
# Rename the variable to make it less confusing
cs2pop_final = cs2pop_final_4km.copy()
pop2cs_final = pop2cs_final_4km.copy()

### Calculating Competition Factor

In [379]:
# Parameter gamma from the formula
gamma = 1

In [398]:
# Creating a mapping from charging station id to its corresponding competition factor
V = {}

for cs_id, pop_id_list in cs2pop_final.items():
    V_j = 0
    for pop_id in pop_id_list:
        pop_value = pop['NTL_value'].values[pop_id]
        cur_dist = int(pre_computed_distance[(pop_id, cs_id)]) / 1000
        V_j += pop_value / (cur_dist ** gamma)
    V[cs_id] = V_j

In [399]:
# Check how many V_j's are 0
cnt = 0
for key, value in V.items():
    if value == 0:
        cnt += 1

print(cnt)
print(cnt/len(V))

67
0.014965378601742237


However, some of the charging stations actually don't have any population point within the threshold distance, which will cause the $V_j$ to be 0 and leads to zero division.
<br>
Well, that doesn't make a difference since no population point will count in those charging stations when computing gravity measure.

### Calculate Accessibility Index using the Improved Gravity Model Formula

In [400]:
alpha = 24 / 6
beta = 24 / 1
theta = 1

In [401]:
charging_station['charging opportunity'] = alpha * charging_station['Num_of_AC'] + beta * charging_station['Num_of_DC']

In [402]:
pop['accessibility'] = np.nan

In [404]:
for pop_id, cs_id_list in pop2cs_final.items():
    A_i = 0
    for cs_id in cs_id_list:
        this_V = V[cs_id]
        this_M = charging_station['charging opportunity'].values[cs_id]
        this_D = int(pre_computed_distance[(pop_id, cs_id)]) / 1000
        A_i += this_M / (this_M * (this_D ** theta))
    pop.loc[pop_id,'accessibility'] = A_i

In [410]:
pop.to_csv('Accessibility.csv')